In [ ]:
import logging
import json
from pathlib import Path
import os

logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s',
                    level=logging.WARNING)

PROJECT_ROOT = Path(os.getcwd()).parents[1].resolve()
DATA_PATH = os.path.join(PROJECT_ROOT, 'data')

MODEL_FILE = 'model_1_'

# load json from file
with open(os.path.join(DATA_PATH, MODEL_FILE + '.json'), 'r') as json_file:
    model_json = json.loads(json_file.read())

In [ ]:
import pandas as pd
from semconstmining.parsing.parser import get_elements_flat
from semconstmining.parsing.model_to_log import check_soundness, create_log
from semconstmining.parsing.conversion.bpmnjsonanalyzer import fromJSON
from semconstmining.parsing.conversion.jsontopetrinetconverter import JsonToPetriNetConverter

f, l, _ = fromJSON(model_json)
elements = pd.DataFrame.from_records(get_elements_flat(model_json, f, l)).set_index('eid')
elements

In [ ]:
converter = JsonToPetriNetConverter()

net, im, fm = converter.convert_from_parsed(f, l)
is_sound = check_soundness(net, im, fm)
print(is_sound)
if is_sound:
    log = create_log(net, im, fm, elements)
    print(log)

In [ ]:
from semconstmining.declare.enums import Template
from semconstmining.declare.declare import Declare


d4py = Declare()
d4py.config.CONSTRAINT_TYPES_TO_IGNORE = [Template.CHAIN_RESPONSE.templ_str,
                                           Template.CHAIN_PRECEDENCE.templ_str, Template.CHAIN_SUCCESSION.templ_str,
                                           Template.CHOICE.templ_str, Template.NOT_CHAIN_RESPONSE.templ_str,
                                             Template.NOT_CHAIN_PRECEDENCE.templ_str]
res = set()
for i, trace in enumerate(log):
    trace.attributes["concept:name"] = str(i)
d4py.log = log
d4py.compute_frequent_itemsets(min_support=0.0, len_itemset=2, algorithm="apriori")
d4py.discovery(consider_vacuity=True, max_declare_cardinality=2, plain=True)
individual_res, associations = d4py.filter_discovery(min_support=0.99, plain=True)
res.update(const for const, checker_results in individual_res.items() if "[]" not in const
           and "[none]" not in const)
res